In [ ]:
import os
import pandas as pd
#os.chidir()

# Read Universalbank.csv ,while reading it replace '?', ',' as NAs
# Check the dimensions and type

In [ ]:
bank=pd.read_csv("UniversalBank.csv",na_values=["?",","])
print(bank.shape)
print(type(bank))

# Print Columns names and check the datatypes of columns(dtypes)

In [ ]:
print(bank.columns)
print(bank.dtypes)

# Check the missing values 

In [ ]:
bank.isnull().sum()

# Impute them with mean or median or mode(It is subjective;fillna())

In [ ]:
#bank['Age'].fillna(bank['Age'].mean(),inplace=True)
bank['Age'].fillna(bank['Age'].mean(),inplace=True)
bank['Experience'].fillna(bank['Experience'].mean(),inplace=True)
bank['Income'].fillna(bank['Income'].mean(),inplace=True)
bank.isnull().sum()

#When you have more columns use the following code
#for column in ['Age', 'Experience', 'Income']:
#    bank[column].fillna(bank[column].mean(), inplace=True)
#If you want to replace with median use median() instead of mean()
#If you want to replace with mode use mode()[0]

In [ ]:
#sample Code for median
print(bank['Age'].median())
#sample code for mode
print(bank['Age'].mode()[0])


# Check the Frequency of Target Varaible(value_counts())

In [ ]:
bank["Personal Loan"].value_counts()

# Drop Id column 

In [ ]:
bank=bank.drop(["ID"],axis=1)

In [ ]:
bank.head(6)

# Check the summary of dataframe(describe())

In [ ]:
bank.describe()

# Do necessary type conversions such as numeric to category(astype('category'))

In [ ]:
bank['Education']=bank['Education'].astype('category')
bank['CD Account']=bank['CD Account'].astype('category')
bank['Online']=bank['Online'].astype('category')
bank['CreditCard']=bank['CreditCard'].astype('category')
bank['Securities Account']=bank['Securities Account'].astype('category')
#Use the following code when you have more aolumns
# for column in ['Education', 'CD Account', 'Online']:
#     bank[column]=bank[column].astype('category')
#bank[bank.select_dtypes(['object']).columns] = bank.select_dtypes(['object']).apply(lambda x: x.astype('category'))
#bank[bank.select_dtypes(['object']).columns] = bank.select_dtypes(['object']).apply(lambda x: x.astype('category'))

# Convert Categorical to dummies(pd.get_dummies)

In [ ]:
bank=pd.get_dummies(bank)
bank.dtypes

In [ ]:
bank.head(5)

# SPLIT THE data in to train and test(use sklearn package)

In [ ]:
from sklearn import preprocessing, metrics, cross_validation
from sklearn.model_selection import train_test_split

In [ ]:
#bank['Personal Loan']=bank['Personal Loan'].astype('category')
# Divide in to train and test
#y=bank["Personal Loan"].astype('category')
y=bank["Personal Loan"]
X=bank.drop('Personal Loan', axis=1)
#from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)  

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Standardize the data (numerical attributes only)( import StandardScaler)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
num_atr=X_train.select_dtypes(['int64']).columns
num_atr.insert(3,'CCAvg')

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train[num_atr])


In [ ]:
X_train[num_atr]=scaler.transform(X_train[num_atr])
X_test[num_atr]=scaler.transform(X_test[num_atr])

In [23]:
print(X_train.shape)
print(X_test.shape)

(4000, 18)
(1000, 18)


# Now It is time for Model BUilding 

# Build Adaboost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
Adaboost_model = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    n_estimators=600,
    learning_rate=1)


Adaboost_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred=Adaboost_model.predict(X_train)
print(confusion_matrix(y_train,y_pred))
accuracy_score(y_train,y_pred)

In [ ]:
y_pred=Adaboost_model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import recall_score
print(recall_score(y_test,y_pred, pos_label=1, average='binary'))

# GridSearch Cross Validation

In [ ]:
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators':[100,150,200], 'learning_rate' : [0.1,0.5,0.9]}
clf = GridSearchCV(AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2)), param_grid, n_jobs=4)
clf.fit(X=X_train, y=y_train)
ada_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

In [ ]:
y_pred=ada_model.predict(X_train)
print(confusion_matrix(y_train,y_pred))
accuracy_score(y_train,y_pred)

In [ ]:
y_pred=ada_model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import recall_score
print(recall_score(y_test,y_pred, pos_label=1, average='binary'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
features = X_train.columns
importances = ada_model.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()


Explore function adaboost regressor
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html

## Build GBM Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=50,learning_rate=0.3)
clf.fit(X=X_train, y=y_train)

In [ ]:
y_pred=clf.predict(X_train)
print(confusion_matrix(y_train,y_pred))
accuracy_score(y_train,y_pred)

In [ ]:
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

# Important Features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
features = X.columns
importances = clf.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

# GridSearch Cross validation

In [2]:
from sklearn.model_selection import GridSearchCV 
 
# Use a grid over parameters of interest
param_grid = { 
           "n_estimators" : [100,150,200,250],
           "max_depth" : [5, 10],
           "learning_rate" : [0.1,0.5,0.9]}
 
CV_GBM = GridSearchCV(estimator=GBM, param_grid=param_grid, cv= 3)
CV_GBM.fit(X=X_train, y=y_train)
#print (CV_GBM.best_score_, CV_GBM.best_params_)

NameError: name 'GBM' is not defined

In [ ]:
y_pred=CV_GBM.predict(X_train)
print(confusion_matrix(y_train,y_pred))
accuracy_score(y_train,y_pred)

In [ ]:
y_pred=CV_GBM.predict(X_test)
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

# xgboostclassifier

In [24]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
y_pred=xgb.predict(X_train)
print(confusion_matrix(y_train,y_pred))
accuracy_score(y_train,y_pred)

In [ ]:
y_pred_test=xgb.predict(X_test)
print(confusion_matrix(y_test,y_pred_test))
accuracy_score(y_test,y_pred_test)